# Rolling aggregation column

In this example, we demonstrate how to define a column using rolling aggregation.

Generally, aggregate columns have to describe two aspects in their definition: 

* how to group elements and 
* how to aggregate data in the groups 

There are two general ways to group elements: 

* Partition elements of one (fact) table into non-overlapping groups using some property with respect to elements of another (group) table which has to be equal for all elements in one group. In SQL, it is implemented as GROUP-BY operation.
* Group elements of a table around elements of this same table using some binary relation among them which can be treated as distance. It is typically implemented in rolling aggregation.

In this example, we describe the second approach where we want to find a sum of sold quantities for the several previous records.

In [13]:
import pandas as pd  # Prosto relies on pandas
import prosto as pr  # Import Prosto toolkit

### Create a new workflow

In [14]:
# Create a workflow
prosto = pr.Prosto("My Prosto Workflow")
# Element name is stored in the id field
print("Workflow name is: ´{}´".format(prosto.id))

Workflow name is: ´My Prosto Workflow´


### Define a source table

We use in-memory data for populating this table and not data from any other table in the workflow.

This table stores sales data. Each time some product is sold a new record is added to this table. Therefore, we assume that all records are ordered according to the time of creation.

In [15]:
sales_data = {
    'product_name': ["beer", "chips", "chips", "beer", "chips"],
    'quantity': [1, 2, 3, 2, 1],
    'price': [10.0, 5.0, 6.0, 15.0, 4.0]
}

sales = prosto.populate(
    # A table definition consists of a name and a list of attributes
    table_name="Sales", attributes=["product_name", "quantity", "price"],

    # Table operation is UDF, input tables and model
    func=lambda **m: pd.DataFrame(data=sales_data), tables=[]
)

### Define an aggregate column

For each record of this table, this column will compute a characteristic, which depends on a number of related records of this same table (not another table). The window parameter determines a group of other (related) records for each record of this table. More specificially, it will select the specified number of previous records.

In [16]:
total = prosto.roll(
    # Column description
    name="quantity_sold", table=sales.id,
    # How to group
    window=3,
    # How to aggregate
    func="lambda x: x.sum()", columns=["quantity"], model={},
)

### Execute the workflow

Above we provided only definitions. In order to really compute the result, we need to execute the workflow. This operation will build a topology (a graph of table and column operations) and then execute these operations according to their dependencies.

In [17]:
prosto.run()

### Explore the result

Once the workflow has been executed, we can read the result data.

In [18]:
table_data = sales.get_data()
table_data.head()

,product_name,quantity,price,quantity_sold
0,beer,1,10.0,None
1,chips,2,5.0,None
2,chips,3,6.0,6
3,beer,2,15.0,7
4,chips,1,4.0,6


### Summary

* A rolling aggregation column groups elements of the table it belongs to (not another table with facts)
* A rolling aggregation column aggregates data by means of an arbitrary user-defined function (in the same way as a groupig columns)
* Like any other column, it can be used in other definitions